In [1]:
import itertools
import string
from tqdm import tqdm_notebook
import cPickle as pickle


A = 'A'
B = 'B'
C = 'C'
D = 'D'
E = 'E'


def generate_orderings(candidates): 
    # returns all linear orderings for a set of candidates
    
    return list(itertools.permutations(candidates))

# function to determine if two lists of orderings are permutations of each other 
def is_voter_permutation(p1, p2):
    _p1 = list(p1)
    _p2 = list(p2)
    for r in _p1: 
        if r in _p2:
            _p2.remove(r)
    return len(_p2) == 0

def is_candidate_permutation_and_anon(p1, p2, permutations):
    
    for perm in permutations:
        _p1 = list(p1)
        _p2 = list(p2)
        for r1 in _p1:
            _r = filter(lambda r2: all([c[0] == perm[c[1]] for c in zip(r1,r2)]),_p2) 
            if len(_r) > 0:
                _p2.remove(_r[0])
        if len(_p2) == 0:
            return True
    return False

    
    return any([all([all([c[0] == perm[c[1]] for c in zip(r[0], r[1])]) for r in zip(p1, p2)]) for perm in permutations])

def is_candidate_permutation(p1, p2, permutations):
    
    return any([all([all([c[0] == perm[c[1]] for c in zip(r[0], r[1])]) for r in zip(p1, p2)]) for perm in permutations])

def generate_permutations(cands, all_permutation):
    return [{c: p[i] for i,c in enumerate(cands)} for  p in all_permutation]

voting_scenarios = list()



In [2]:
# Generate Pointed Profiles

import itertools
import string
from tqdm import tqdm_notebook
import cPickle as pickle
import copy

A = 'A'
B = 'B'
C = 'C'
D = 'D'
E = 'E'



voting_scenarios = list()

voting_sizes = [(4,4), (3,8)]
for vs in tqdm_notebook(voting_sizes,desc="Voting Sits."): 
    
    print 
    print vs
    
    num_candidates = vs[0]
    num_voters = vs[1]
    candidates =  list(string.ascii_uppercase[0:num_candidates])
    voter_names = list(string.printable[1:num_voters + 1])
    
    all_orderings = generate_orderings(candidates)    
    
    permutations = generate_permutations(candidates, all_orderings)
    
    print "Linear rankings:  {}".format(len(all_orderings))

    all_profiles = list(itertools.product(*[all_orderings]*len(voter_names)))
    print "Profiles: {}".format(len(all_profiles))

    anon_classes = list()
    reduced_by_anonymity_profiles = list()
    for _p in tqdm_notebook(all_profiles, desc="All profiles", leave=False):
        _class = filter(lambda _prf: is_voter_permutation(_p, _prf[0]), anon_classes)
        if len(_class) == 0:

            a_class = [_p]
            anon_classes.append(a_class)
        else:
            _class[0].append(_p)
        
    # record anon_classes
    _anon_classes = copy.deepcopy(anon_classes)
    
    anon_neutral_classes = list()
    for _ac in anon_classes:
        
        _class = filter(lambda _anc: is_candidate_permutation_and_anon(_ac[0], _anc[0], permutations), 
                        anon_neutral_classes)

        if len(_class) == 0:
            an_class = _ac
            anon_neutral_classes.append(an_class)
        else:
            _class[0] += _ac
            
                
    aecs = [aec[0] for aec in _anon_classes]
    anecs =  [anec[0] for anec in anon_neutral_classes]
    
    print "\nSanity check: len(aecs) = {}, len(anecs) = {}".format(len(aecs), len(anecs))
    voting_scenarios.append({"num_candidates": num_candidates,
                             "num_voters": num_voters, 
                             "candidates": candidates,
                             "voter_names": voter_names,
                             "all_orderings": all_orderings,
                             "profiles": all_profiles,
                             "pointed_profiles": [(p,vname) for vname in voter_names for p in all_profiles],                             
                             "anon_classes": _anon_classes,
                             "aecs": aecs,
                             "pointed_aecs": [(p,vname) for vname in voter_names for p in aecs],
                             "anon_neut_classes": anon_neutral_classes,
                             "anecs": anecs,
                             "pointed_anecs": [(p,vname) for vname in voter_names for p in anecs]
                            })

    



#for vs in voting_scenarios:
#    print "for {} cand, {} voters there are {} pointed profiles".format(vs["num_candidates"], vs["num_voters"],len(vs["pointed_profiles"]))
#print "\n\n"
    print "writing pickle file...."
    pickle.dump(voting_scenarios, open("voting_scenarios3.pkl","w"))

print "done."


(4, 4)
Linear rankings:  24
Profiles: 331776



Sanity check: len(aecs) = 17550, len(anecs) = 762
writing pickle file....

(3, 8)
Linear rankings:  6
Profiles: 1679616



Sanity check: len(aecs) = 1287, len(anecs) = 222
writing pickle file....

done.
